# Bain Strategic Challenge

In [28]:
import pandas as pd
import numpy as np
import matplotlib as plt
import time
import datetime

In [43]:
'''
Unfortunately, the way the destinated_area columns is made makes it impossible to simply use
pd.read_csv('fn')
as it is not able to interpret lines like
- number,number
- number,
- ,
simultaneously.

This can be remedied by manually extracting the data from the file and treating some edge cases.
We also take the chance to convert the dates to a Timestamp, which allows for some better ease of use later.
'''


with open('historical-database.csv', 'r') as f:
    data = f.read()

data = data.split('\n')
# Separating columns manually
columns = data[0]
data = data[1:-1]
# Remove ending comma, then split by semi-colon
columns = columns[:-1].split(';')

def process_line(line: str) -> list:
    entries = line.split(';')
    if ',' in entries[-1] and len(entries[-1])>1:
        entries[-1] = float(entries[-1].replace(',','.',-1))
    elif len(entries[-1]) <= 1:
        entries[-1] = None
    else:
        entries[-1] = float(entries[-1])
    # Convert to python-friendly date format.
    entries[0] = datetime.datetime.strptime(entries[0],"%d/%m/%Y")
    return entries
data = list(map(process_line, data))
df = pd.DataFrame(data=data, columns=columns)
df


,year,city_code,product_type,product,destinated_area
0,1974-01-01,960b4f2c94a2fb2c,temporary,Others,0.0000
1,1974-01-01,746cc42bfb8f6b62,temporary,Others,0.0000
2,1974-01-01,6cce2bf873870afc,temporary,Others,0.0000
3,1974-01-01,4de42e351006a2ae,temporary,Others,0.0000
4,1974-01-01,5b6072f8f6d37acc,temporary,Others,0.0000
...,...,...,...,...,...
73739,2013-01-01,74d75dde6dc4a5ec,pasture,Livestock,150866.5644
73740,2014-01-01,74d75dde6dc4a5ec,pasture,Livestock,152446.3960
73741,2015-01-01,74d75dde6dc4a5ec,pasture,Livestock,154681.0205
73742,2016-01-01,74d75dde6dc4a5ec,pasture,Livestock,164706.2778
